In [1]:
data_var = '2024-01-22'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [ ]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
13006,2024-01-21,Noruega Blno,08:00,Tromso,Nidaros Jets,1.04,9.10,160.5,1.85,1.85,-18.5,2.08,1.66,2311z3GS,0.961538,0.109890,0.540541,0.540541,0.071429,131.840,54.526964,0.413584,1.8,1.643168,0.912871,89.70,1.626,0.610885,0.375698,18.0,209.380,108.744480,0.519364,0.6,1.341641,2.236068,383.64,3.098,1.718494,0.554711,-83.0,69,69,1.30,5.56,130.440,0.000,1.124118,0.000000,0.158815,-1.66,-0.332,-0.120482,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
13007,2024-01-21,Japão B2.League,02:10,Niigata,Shiga,6.47,1.09,160.5,1.85,1.85,12.5,2.00,1.72,EidVESiC,0.154560,0.917431,0.540541,0.540541,0.071991,250.700,52.438753,0.209169,1.2,1.643168,1.369306,204.48,3.246,0.588668,0.181352,-10.0,110.340,9.318050,0.084449,3.0,0.000000,0.000000,100.05,1.200,0.041833,0.034861,51.0,72,87,2.84,1.15,397.810,114.662,1.006411,0.000000,0.106446,-2.03,-0.406,-13.472906,0.000000,0.0,0.000000,-0.05,-0.010,-9.000000,0.000000,0.0,0.000000
13008,2024-01-21,Japão B2.League,02:00,Yamagata Wyverns,Aomori,1.43,2.68,167.5,1.83,1.87,-6.5,2.03,1.70,W80AJjws,0.699301,0.373134,0.546448,0.534759,0.072435,222.062,108.722196,0.489603,3.0,0.000000,0.000000,257.50,2.526,1.306629,0.517272,35.0,136.364,26.608541,0.195129,0.6,1.341641,2.236068,164.88,1.774,0.407529,0.229723,-34.0,103,72,2.50,2.29,303.176,130.788,0.430114,0.015289,0.125118,-2.54,-0.508,-0.846457,0.000000,0.0,0.000000,-2.19,-0.438,-3.835616,0.560725,0.5,-0.060725
13009,2024-01-21,Japão B2.League,03:00,Altiri Chiba,Fukuoka,1.14,5.08,155.5,1.80,1.86,-12.5,2.01,1.71,0lhRF8x6,0.877193,0.196850,0.555556,0.537634,0.074043,101.182,13.953950,0.137909,2.4,1.341641,0.559017,94.60,1.132,0.067602,0.059719,57.0,102.412,7.904244,0.077181,1.2,1.643168,1.369306,99.40,1.382,0.143422,0.103779,-18.0,86,71,1.10,1.40,105.582,127.074,0.895820,0.023184,0.114049,0.62,0.124,1.129032,0.000000,0.0,0.000000,-0.15,-0.030,-136.000000,0.000000,0.0,0.000000
13010,2024-01-21,Japão B2.League,03:00,Shizuoka,Nara,1.65,2.15,152.5,1.87,1.83,-3.5,2.00,1.72,AaaEIAhm,0.606061,0.465116,0.534759,0.546448,0.071177,222.526,104.840409,0.471138,0.6,1.341641,2.236068,137.08,2.742,1.122439,0.409351,-42.0,199.114,51.456473,0.258427,2.4,1.341641,0.559017,193.38,2.528,0.657092,0.259926,-7.0,92,66,1.49,2.93,156.768,241.106,0.186081,0.015289,0.106446,-0.43,-0.086,-7.558140,0.000000,0.0,0.000000,-2.44,-0.488,-2.356557,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13206,2024-01-21,Eua Ncaa,17:00,Utah Utes,Oregon,1.51,2.61,152.5,1.80,1.91,3.5,1.25,0.00,rXwBzQpE,0.662252,0.383142,0.555556,0.523560,0.045393,164.540,137.939889,0.838337,1.2,1.643168,1.369306,77.70,2.206,1.917767,0.869341,36.0,145.512,52.186363,0.358640,2.4,1.341641,0.559017,221.90,1.970,0.756968,0.384248,6.0,74,70,1.05,3.17,100.690,134.598,0.377581,0.041931,1.414214,0.92,0.184,2.771739,0.812913,0.9,0.087087,-1.75,-0.350,-4.600000,0.000000,0.0,0.000000
13207,2024-01-21,Eua Ncaa,17:00,UTSA Roadrunners,Florida Atlantic,9.20,1.06,160.5,1.80,1.91,18.5,1.60,0.00,CYbW2DtJ,0.108696,0.943396,0.555556,0.523560,0.052092,507.664,570.493782,1.123763,0.6,1.341641,2.236068,271.44

## Apostas do dia

### Back Home V3 (QCut)

In [ ]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
13010,03:00,Japão B2.League,Shizuoka,Nara,1.65,Back Home
13039,11:00,Itália Série A1 Feminina,Sassari F,San Giovanni F,1.99,Back Home
13099,03:30,Taiwan P. League+,Hsinchu Jko Lioneers,Taoyuan Pauian Pilots,1.59,Back Home
13109,00:00,Austrália Nbl,Sydney Kings,Perth Wildcats,1.69,Back Home
13155,14:30,Alemanha Bbl,Ludwigsburg,Vechta,1.56,Back Home
13202,16:00,Eua Ncaa,Sacred Heart,Wagner,1.55,Back Home
13204,16:00,Eua Ncaa,St. Peters,Canisius,1.55,Back Home
